In [1]:
%env FLASK_ENV=development

env: FLASK_ENV=development


In [2]:
from dotenv import load_dotenv

load_dotenv()

from mongoengine import connect, disconnect

from settings import DevelopmentConfig

from recommender import User
from recommender.models import Service
from recommender.engines.nlp_embedders.embedders import services2tensor, users2tensor

/home/jan/.local/share/virtualenvs/recommender-system-r71yvAN0/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
2022-04-28 09:02:19.188598: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-04-28 09:02:19.188695: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-Jan
2022-04-28 09:02:19.188723: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-Jan
2022-04-28 09:02:19.188919: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.103.1
2022-04-28 09:02:19.188981: I ten

In [5]:
disconnect()
connection = connect(host=DevelopmentConfig.MONGODB_HOST)

### Object to Vector Concept

object -> vec

object has known structure so it's easier than variable structure. We can do one net architecture for given structure :)

Input data models is:
List of dicts of lists of dicts of strings (there is no error in this phrase XD)

### Example of data
```
batch = [
    object1 = {
        "field1": [
            {
                "sub_field1_1": "val1",
                "sub_field1_2": "val2"
            },
            {
                "sub_field1_1": "val3",
                "sub_field1_2": "val4"
            }
        ],
        "field2": [
            {
                "sub_field2_1": "val5",
                "sub_field2_2": "val6"
            },
            {
                "sub_field2_1": "val7",
                "sub_field2_2": "val8"
            }
        ],
    },
    object2 = {
        "field1": [
            {
                "sub_field1_1": "val9",
                "sub_field1_2": "val10"
            },
            {
                "sub_field1_1": "val11",
                "sub_field1_2": "val12"
            }
        ],
        "field2": [
            {
                "sub_field2_1": "val13",
                "sub_field2_2": "val14"
            },
            {
                "sub_field2_1": "val15",
                "sub_field2_2": "val16"
            }
        ],
    },
]
```
### Next step

```
batch = [
    object1 = {
        "sub_field1_1": ["val1", "val3"],
        "sub_field1_2": ["val2", "val4"],
        "sub_field2_1": ["val5", "val7"],
        "sub_field2_2": ["val6", "val8"],
    },
    object2 = {
        "sub_field1_1": ["val9", "val11"],
        "sub_field1_2": ["val10", "val12"],
        "sub_field2_1": ["val13", "val15"],
        "sub_field2_2": ["val14", "val16"],
    },
]
```

### Next step

```
batch_sub_field1_1 = [
    ["val1", "val3"],
    ["val9", "val11"]
]

batch_sub_field1_2 = [
    ["val2", "val4"],
    ["val10", "val12"]
]

batch_sub_field2_1 = [
    ["val5", "val7"],
    ["val13", "val15"]
]

batch_sub_field2_2 = [
    ["val6", "val8"],
    ["val14", "val16"]
]
```

### Next step (target form)

```
batch_sub_field1_1 = [
    [vec1, vec3],
    [vec9, vec11]
]

batch_sub_field1_2 = [
    [vec2, vec4],
    [vec10, vec12]
]

batch_sub_field2_1 = [
    [vec5, vec7],
    [vec13, vec15]
]

batch_sub_field2_2 = [
    [vec6, vec8],
    [vec14, vec15]
]
```

objects can be nested (in the limited way) so we have to have ways of embedding:
- basic types: strings encoded with text2vec (done with [Universal Sentence Encoder](https://github.com/MartinoMensio/spacy-universal-sentence-encoder))
- lists: Lists need flattening of the temporal dimension (using e.g.: mean() ) - done using so called handlers
- dicts: no problemo, we split them into separate batches - done using so called handlers

Theoretical time complexity is linear :)

### Execution time tests

In [7]:
%timeit services2tensor(Service.objects[:1])

27.3 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit services2tensor(Service.objects)

10.1 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
services = services2tensor(Service.objects[:100])

In [13]:
services.shape

torch.Size([100, 9728])

In [11]:
%timeit users2tensor(User.objects[:1])

2.62 ms ± 33.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%timeit users2tensor(User.objects)

1.81 s ± 39.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Discussion

Now we can embedd Users, Services or almost any other objects without any embedders training and whole dataset collection in real time. We can do it to new objects that we've never seen before. We are not constrained by possible values of object fields or by old DB dumps or by existing or not existing IDs. Data can be in the any language!!!

It's a little bit too good to be true, so extensive tests of recommendation power needed.

NLP part could and should be vastly extended and improved, for now there is necessary minimum and no more.

Some mechanism for dimensinality reduction should be proposed, e.g.:
- polling in the first layers of networks that deal with user/services
- NLP-based sentences concatenation

If above ideas works (tests needed) the big part of the recommender system that deals with data preparation could be removed.

## Conclusion

> Is it possible to ingest MP DB changes as events in the Recommender and use them for online training and how exactly should it be done?
> -- <cite>[Data Ingestion Redesign Issue #348](https://github.com/cyfronet-fid/recommender-system/issues/348)</cite>

> ### Yes, it's possible and can be done using already implemented technique presented above

What the excellent PR should have:
- implementation
- tests
- refactoring done
- docstrings, examples, readme
- typing
- black, pylint run
- theoretical time and memory complexity
- practical time and memory complexity: profiling
- usage of multiprocesing
- usage of loggers, verbosity
- progress bars if needed